In [74]:
#Uploading required packages
import numpy as np
from scipy.stats import gamma
from scipy.stats import uniform as unif
from scipy.stats import multivariate_normal as mvtnorm
from scipy.stats import bernoulli
from numba import jit
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Indian Buffet Process Function

def sampleIBP(alpha, num_objects):  
    # Initializing storage for results
    result = np.zeros([num_objects, 1000])
    # Draw from the prior for alpha
    t = np.random.poisson(alpha)
    # Filling in first row of result matrix
    result[0, 0:t] = np.ones(t) #changed form np.ones([1, t])
    # Initializing K+
    K_plus = t
    
    for i in range(1, num_objects):
        for j in range(0, K_plus):
            p = np.array([np.log(np.sum(result[0:i,j])) - np.log(i+1), 
                          np.log(i+1 - np.sum(result[0:i, j])) - np.log(i+1)])
            p = np.exp(p - max(p))

            if(np.random.uniform() < p[0]/np.sum(p)):
                result[i, j] = 1
            else:
                result[i, j] = 0
        t = np.random.poisson(alpha/(i+1))
        x = K_plus + 1
        y = K_plus + t
        result[i, (x-1):y] = np.ones(t) #changed form np.ones([1, t])
        K_plus = K_plus+t
    result = result[:, 0:K_plus]
    
    return list([result, K_plus])

In [63]:
#Data Simulation

#Latent Features
W = np.array([[0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]])


#Each image in our simulated data set is the superposition of four base images#
# Number of images/ data points
num_objects=100

#Dimension of image (6x6)
object_dim  = 6*6

#Covariance matrix for images/ white noise
sigma_x_orig = 0.5
I = sigma_x_orig * np.identity(object_dim)

#z_i - binary feature matrix (1 x 4) - each entry set to 1 with probability 0.5 and 0 otherwise#
#x is data variable - each row correspondes to a superimposed built from a random combination of latent features#
#with white noise added - x is built with multivariate gaussian#
image_data = np.zeros((100,36))
z_org = np.zeros((100,4))

for i in range(0,num_objects):
    z_org[i,:] = np.array([bernoulli.rvs(p=0.5, size=4)])
    image_data[i,:] = np.dot(z_org[i,:],W) + np.random.normal(0,1, (1,object_dim)).dot(I) 

In [75]:
## Optimized likelihood
# This function return the log likelihood
def likelihood_opt(X, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    #Calculate M
    M = np.dot(Z.T, Z) + (sigma_X**2/sigma_A**2)*np.eye(K_plus)
    
    part1 = (-1)*num_objects*(0.5*object_dim)*np.log(2*np.pi)
    part2 = (-1)*(num_objects-K_plus)* object_dim *np.log(sigma_X) 
    part3 = (-1)*object_dim*K_plus*np.log(sigma_A) 
    part4 = (-1)*(0.5*object_dim)* np.log(np.linalg.det(M)) 
    part5 = (-1/(2*sigma_X**2)) * np.trace(np.dot(np.dot(X.T,(np.identity(num_objects) - np.dot(np.dot(Z,np.linalg.inv(M)),Z.T))),X))
    total = part1+part2+part3+part4+part5
    return(total)

Naive
====

In [78]:
%%timeit
data = image_data

#Set number of iterations
E = 100

#Set truncation limit for max number of sampled latent features
K_inf = 20

#Set storage arrays for sampled parameters
chain_Z = np.zeros([E, num_objects, K_inf])
chain_K = np.zeros([E, 1])
chain_sigma_X = np.zeros([E, 1])
chain_sigma_A = np.zeros([E, 1])
chain_alpha = np.zeros([E, 1])

#Initialize parameter values
sigma_X = 1
sigma_A = 1
alpha = 1
num_object= np.shape(data)[0]
object_dim = np.shape(data)[1]
[Z, K_plus] = sampleIBP(alpha, num_objects)

#Compute Harmonic Number
HN = 0
for i in range(0, num_objects):
    HN = HN + 1/(i+1)

for e in range(0, E):

    #Store sampled values
    chain_Z[e, :, 0:K_plus] = Z[:, 0:K_plus]
    chain_K[e] = K_plus
    chain_sigma_X[e] = sigma_X
    chain_sigma_A[e] = sigma_A
    chain_alpha[e] = alpha
    
    if (e%100==0):
        print(e)
    
    print("At iteration", e, ": K_plus is", K_plus, ", alpha is", alpha) 

    #Generate a new value for Z[i,k] and accept by Metropolis
    for i in range(0, num_objects):
        #First we remove singular features if any
        for k in range(0, K_plus):
            if (k>=K_plus):
                break
            if(Z[i, k] > 0):
                if (np.sum(Z[:, k]) - Z[i, k]) <= 0: 
                    Z[i, k] = 0
                    Z[:, k:(K_plus - 1)] = Z[:, (k+1):K_plus]
                    K_plus = K_plus - 1
                    Z = Z[:, 0:K_plus]
                    continue
            #Compute conditional distribution for current cell
            P=np.zeros(2)

            Z[i,k]=1
            P[0] = likelihood_opt(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(np.sum(Z[:, k]) - Z[i,k]) - np.log(num_objects)

            Z[i,k]=0
            P[1] = likelihood_opt(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - np.sum(Z[:,k])) - np.log(num_objects)

            P = np.exp(P - np.max(P))

            #Sample from conditional(sampling previously sampled dishes)
            if np.random.uniform(0,1) < (P[0]/(np.sum(P))):
                Z[i, k] = 1
            else:
                Z[i, k] = 0
                
        #Sample new dishes by Metropolis
        trunc = np.zeros(5)
        alpha_N = alpha/num_objects

        for k_i in range(0,5):
            Z_temp = Z
            if k_i>0:
                newcol = np.zeros((num_objects, k_i))
                newcol[i,:] = 1 
                Z_temp = np.column_stack((Z_temp, newcol))
            trunc[k_i] = k_i * np.log(alpha_N) - alpha_N - np.log(np.math.factorial(k_i)) + likelihood_opt(data, Z_temp,sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

        trunc = np.exp(trunc - np.max(trunc))
        trunc = trunc/np.sum(trunc)

        p = np.random.uniform(0,1)
        t = 0
        new_dishes = 0

        for k_i in range(0,5):
            t = t + trunc[k_i]
            if p < t:
                new_dishes = k_i
                break

        if(new_dishes > 0):
            newcol = np.zeros((num_objects, new_dishes))
            newcol[i,:] = 1
            Z = np.column_stack((Z, newcol))
        K_plus = K_plus + new_dishes

    #Sample sigma_X and sigma_A through Metropolis
    lik_curr = likelihood_opt(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim)

    if np.random.uniform(0,1) < 0.5:
        sigma_X_new = sigma_X - np.random.uniform(0,1)/20
    else:
        sigma_X_new = sigma_X + np.random.uniform(0,1)/20

    lik_new_X = likelihood_opt(data, Z, sigma_A, sigma_X_new, K_plus, num_objects, object_dim)

    acc_X = np.exp(min(0, lik_new_X - lik_curr))

    if np.random.uniform(0,1) < 0.5:
        sigma_A_new = sigma_A - np.random.uniform(0,1)/20
    else:
        sigma_A_new = sigma_A + np.random.uniform(0,1)/20

    lik_new_A = likelihood_opt(data, Z, sigma_A_new, sigma_X, K_plus, num_objects, object_dim)

    acc_A = np.exp(min(0, lik_new_A - lik_curr))

    if np.random.uniform(0,1) < acc_X:
        sigma_X = sigma_X_new
    if np.random.uniform(0,1) < acc_A:
        sigma_A = sigma_A_new

    #Sample alpha via Gibbs
    alpha = np.random.gamma(1 + K_plus, 1/(1+HN))

print("Complete")

0
At iteration 0 : K_plus is 4 , alpha is 1
At iteration 1 : K_plus is 1 , alpha is 0.13255892289504143
At iteration 2 : K_plus is 1 , alpha is 0.14621789608852692
At iteration 3 : K_plus is 1 , alpha is 0.01612633642796229
At iteration 4 : K_plus is 1 , alpha is 0.2631909358119492
At iteration 5 : K_plus is 1 , alpha is 0.7864013241977086
At iteration 6 : K_plus is 1 , alpha is 0.19479854027448473
At iteration 7 : K_plus is 1 , alpha is 0.5337821575575145
At iteration 8 : K_plus is 1 , alpha is 0.2998484450498264
At iteration 9 : K_plus is 1 , alpha is 0.20351729922006226
At iteration 10 : K_plus is 1 , alpha is 0.1136365855608684
At iteration 11 : K_plus is 1 , alpha is 0.2386403799970838
At iteration 12 : K_plus is 1 , alpha is 0.48457847472861626
At iteration 13 : K_plus is 1 , alpha is 0.2526790949246784
At iteration 14 : K_plus is 1 , alpha is 0.3257034913155807
At iteration 15 : K_plus is 1 , alpha is 0.9571807201505018
At iteration 16 : K_plus is 1 , alpha is 0.299665237173367


Cython
====

In [4]:
%load_ext cython

In [6]:
%%cython -a

import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
def matrix_multiply1(double[:,:] u, double[:, :] v):
    cdef int i, j, k
    cdef int m, n, p
    
    cdef double[:, :] res

    m = u.shape[0]
    n = u.shape[1]
    p = v.shape[1]
    
    res = np.zeros([m, p])

    with cython.nogil:
        for i in range(m):
            for j in range(p):
                res[i,j] = 0
                for k in range(n):
                    res[i,j] += u[i,k] * v[k,j]

In [18]:
%%cython -a

import numpy as np
cimport cython
## Optimized likelihood
# This function return the log likelihood
def likelihood_opt1(double[:,:] X, double[:,:] Z, double sigma_A, double sigma_X, int K_plus, int num_objects, int object_dim):
    
    cdef double[:,:] M
    cdef double part1, part2, part3, part4, part5
        
    #Calculate M
    M = np.dot(Z.T, Z) + (sigma_X**2/sigma_A**2)*np.eye(K_plus)
    
    part1 = (-1)*num_objects*(0.5*object_dim)*np.log(2*np.pi)
    part2 = (-1)*(num_objects-K_plus)* object_dim *np.log(sigma_X) 
    part3 = (-1)*object_dim*K_plus*np.log(sigma_A) 
    part4 = (-1)*(0.5*object_dim)* np.log(np.linalg.det(M)) 
    part5 = (-1/(2*sigma_X**2)) * np.trace(np.dot(np.dot(X.T,(np.identity(num_objects) - np.dot(np.dot(Z,np.linalg.inv(M)),Z.T))),X))
    total = part1+part2+part3+part4+part5
    return(total)

In [23]:
%%timeit

data = image_data

#Set number of iterations
E = 1000

#Set truncation limit for max number of sampled latent features
K_inf = 20

#Set storage arrays for sampled parameters
chain_Z = np.zeros([E, num_objects, K_inf])
chain_K = np.zeros([E, 1])
chain_sigma_X = np.zeros([E, 1])
chain_sigma_A = np.zeros([E, 1])
chain_alpha = np.zeros([E, 1])

#Initialize parameter values
sigma_X = 1
sigma_A = 1
alpha = 1
num_object= np.shape(data)[0]
object_dim = np.shape(data)[1]
[Z, K_plus] = sampleIBP(alpha, num_objects)

#Compute Harmonic Number
HN = 0
for i in range(0, num_objects):
    HN = HN + 1/(i+1)

for e in range(0, E):

    #Store sampled values
    chain_Z[e, :, 0:K_plus] = Z[:, 0:K_plus]
    chain_K[e] = K_plus
    chain_sigma_X[e] = sigma_X
    chain_sigma_A[e] = sigma_A
    chain_alpha[e] = alpha
    
    if (e%100==0):
        print(e)
    
    print("At iteration", e, ": K_plus is", K_plus, ", alpha is", alpha) 

    #Generate a new value for Z[i,k] and accept by Metropolis
    for i in range(0, num_objects):
        #First we remove singular features if any
        for k in range(0, K_plus):
            if (k>=K_plus):
                break
            if(Z[i, k] > 0):
                if (np.sum(Z[:, k]) - Z[i, k]) <= 0: 
                    Z[i, k] = 0
                    Z[:, k:(K_plus - 1)] = Z[:, (k+1):K_plus]
                    K_plus = K_plus - 1
                    Z = Z[:, 0:K_plus]
                    continue
            #Compute conditional distribution for current cell
            P=np.zeros(2)

            Z[i,k]=1
            P[0] = likelihood_opt1(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(np.sum(Z[:, k]) - Z[i,k]) - np.log(num_objects)

            Z[i,k]=0
            P[1] = likelihood_opt1(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - np.sum(Z[:,k])) - np.log(num_objects)

            P = np.exp(P - np.max(P))

            #Sample from conditional(sampling previously sampled dishes)
            if np.random.uniform(0,1) < (P[0]/(np.sum(P))):
                Z[i, k] = 1
            else:
                Z[i, k] = 0
                
        #Sample new dishes by Metropolis
        trunc = np.zeros(5)
        alpha_N = alpha/num_objects

        for k_i in range(0,5):
            Z_temp = Z
            if k_i>0:
                newcol = np.zeros((num_objects, k_i))
                newcol[i,:] = 1 
                Z_temp = np.column_stack((Z_temp, newcol))
            trunc[k_i] = k_i * np.log(alpha_N) - alpha_N - np.log(np.math.factorial(k_i)) + likelihood_opt1(data, Z_temp,sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

        trunc = np.exp(trunc - np.max(trunc))
        trunc = trunc/np.sum(trunc)

        p = np.random.uniform(0,1)
        t = 0
        new_dishes = 0

        for k_i in range(0,5):
            t = t + trunc[k_i]
            if p < t:
                new_dishes = k_i
                break

        if(new_dishes > 0):
            newcol = np.zeros((num_objects, new_dishes))
            newcol[i,:] = 1
            Z = np.column_stack((Z, newcol))
        K_plus = K_plus + new_dishes

    #Sample sigma_X and sigma_A through Metropolis
    lik_curr = likelihood_opt1(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim)

    if np.random.uniform(0,1) < 0.5:
        sigma_X_new = sigma_X - np.random.uniform(0,1)/20
    else:
        sigma_X_new = sigma_X + np.random.uniform(0,1)/20

    lik_new_X = likelihood_opt1(data, Z, sigma_A, sigma_X_new, K_plus, num_objects, object_dim)

    acc_X = np.exp(min(0, lik_new_X - lik_curr))

    if np.random.uniform(0,1) < 0.5:
        sigma_A_new = sigma_A - np.random.uniform(0,1)/20
    else:
        sigma_A_new = sigma_A + np.random.uniform(0,1)/20

    lik_new_A = likelihood_opt1(data, Z, sigma_A_new, sigma_X, K_plus, num_objects, object_dim)

    acc_A = np.exp(min(0, lik_new_A - lik_curr))

    if np.random.uniform(0,1) < acc_X:
        sigma_X = sigma_X_new
    if np.random.uniform(0,1) < acc_A:
        sigma_A = sigma_A_new

    #Sample alpha via Gibbs
    alpha = np.random.gamma(1 + K_plus, 1/(1+HN))

print("Complete")

0
At iteration 0 : K_plus is 11 , alpha is 1
At iteration 1 : K_plus is 5 , alpha is 0.6058680513757068
At iteration 2 : K_plus is 4 , alpha is 0.6124352357245961
At iteration 3 : K_plus is 3 , alpha is 0.15924129519611616
At iteration 4 : K_plus is 3 , alpha is 1.1749654422058566
At iteration 5 : K_plus is 3 , alpha is 0.9074787405051914
At iteration 6 : K_plus is 3 , alpha is 0.6776310739594353
At iteration 7 : K_plus is 3 , alpha is 0.40341768408982587
At iteration 8 : K_plus is 3 , alpha is 0.38258721248606836
At iteration 9 : K_plus is 3 , alpha is 0.7230686998066929
At iteration 10 : K_plus is 3 , alpha is 0.5689203429705895
At iteration 11 : K_plus is 3 , alpha is 0.4119164724596606
At iteration 12 : K_plus is 3 , alpha is 0.9118176478996056
At iteration 13 : K_plus is 3 , alpha is 0.5298700055205547
At iteration 14 : K_plus is 3 , alpha is 0.9260530718863433
At iteration 15 : K_plus is 3 , alpha is 0.7507087050345781
At iteration 16 : K_plus is 3 , alpha is 0.5432965887297747
A

KeyboardInterrupt: 

In [91]:
%%cython -a

import numpy as np
from scipy.stats import gamma
from scipy.stats import uniform as unif
from scipy.stats import multivariate_normal as mvtnorm
from scipy.stats import bernoulli
cimport cython

def sampleIBP(double alpha, int num_objects):  
    
    cdef int i, j, k
    cdef int K_plus
    # Initializing storage for results
    result = np.zeros([num_objects, 1000])
    # Draw from the prior for alpha
    t = np.random.poisson(alpha)
    # Filling in first row of result matrix
    result[0, 0:t] = np.ones(t) #changed form np.ones([1, t])
    # Initializing K+
    K_plus = t
    
    for i in range(1, num_objects):
        for j in range(0, K_plus):
            p = np.array([np.log(np.sum(result[0:i,j])) - np.log(i+1), 
                          np.log(i+1 - np.sum(result[0:i, j])) - np.log(i+1)])
            p = np.exp(p - max(p))

            if(np.random.uniform() < p[0]/np.sum(p)):
                result[i, j] = 1
            else:
                result[i, j] = 0
        t = np.random.poisson(alpha/(i+1))
        x = K_plus + 1
        y = K_plus + t
        result[i, (x-1):y] = np.ones(t) #changed form np.ones([1, t])
        K_plus = K_plus+t
    result = result[:, 0:K_plus]
    
    return list([result, K_plus])

def likelihood_opt1(double[:,:] X, double[:,:] Z, double sigma_A, double sigma_X, int K_plus, int num_objects, int object_dim):
    
    cdef double[:,:] M
    cdef double part1, part2, part3, part4, part5
        
    #Calculate M
    M = np.dot(Z.T, Z) + (sigma_X**2/sigma_A**2)*np.eye(K_plus)
    
    part1 = (-1)*num_objects*(0.5*object_dim)*np.log(2*np.pi)
    part2 = (-1)*(num_objects-K_plus)* object_dim *np.log(sigma_X) 
    part3 = (-1)*object_dim*K_plus*np.log(sigma_A) 
    part4 = (-1)*(0.5*object_dim)* np.log(np.linalg.det(M)) 
    part5 = (-1/(2*sigma_X**2)) * np.trace(np.dot(np.dot(X.T,(np.identity(num_objects) - np.dot(np.dot(Z,np.linalg.inv(M)),Z.T))),X))
    total = part1+part2+part3+part4+part5
    return(total)

def sampler(image_data, num_objects, object_dim):
    
    cdef int i, e, k_i, k, K_plus, E
    cdef double HN, alpha_N, p
    cdef double[:, :] Z, Z_temp
    cdef double alpha
    #cdef double[:] trunc, chain_K
    
    data = image_data

    #Set number of iterations
    E = 100

    #Set truncation limit for max number of sampled latent features
    K_inf = 20

    #Set storage arrays for sampled parameters
    chain_Z = np.zeros([E, num_objects, K_inf])
    chain_K = np.zeros([E, 1])
    chain_sigma_X = np.zeros([E, 1])
    chain_sigma_A = np.zeros([E, 1])
    chain_alpha = np.zeros([E, 1])

    #Initialize parameter values
    sigma_X = 1
    sigma_A = 1
    alpha = 1
    num_object= np.shape(data)[0]
    object_dim = np.shape(data)[1]
    [Z, K_plus] = sampleIBP(alpha, num_objects)

    #Compute Harmonic Number
    HN = 0
    for i in range(0, num_objects):
        HN = HN + 1/(i+1)

    for e in range(0, E):

        #Store sampled values
        chain_Z[e, :, 0:K_plus] = Z[:, 0:K_plus]
        chain_K[e] = K_plus
        chain_sigma_X[e] = sigma_X
        chain_sigma_A[e] = sigma_A
        chain_alpha[e] = alpha

        if (e%100==0):
            print(e)

        print("At iteration", e, ": K_plus is", K_plus, ", alpha is", alpha) 

        #Generate a new value for Z[i,k] and accept by Metropolis
        for i in range(0, num_objects):
            #First we remove singular features if any
            for k in range(0, K_plus):
                if (k>=K_plus):
                    break
                if(Z[i, k] > 0):
                    if (np.sum(Z[:, k]) - Z[i, k]) <= 0: 
                        Z[i, k] = 0
                        Z[:, k:(K_plus - 1)] = Z[:, (k+1):K_plus]
                        K_plus = K_plus - 1
                        Z = Z[:, 0:K_plus]
                        continue
                #Compute conditional distribution for current cell
                P=np.zeros(2)

                Z[i,k]=1
                P[0] = likelihood_opt1(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(np.sum(Z[:, k]) - Z[i,k]) - np.log(num_objects)

                Z[i,k]=0
                P[1] = likelihood_opt1(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - np.sum(Z[:,k])) - np.log(num_objects)

                P = np.exp(P - np.max(P))

                #Sample from conditional(sampling previously sampled dishes)
                if np.random.uniform(0,1) < (P[0]/(np.sum(P))):
                    Z[i, k] = 1
                else:
                    Z[i, k] = 0

            #Sample new dishes by Metropolis
            trunc = np.zeros(5)
            alpha_N = alpha/num_objects

            for k_i in range(0,5):
                Z_temp = Z
                if k_i>0:
                    newcol = np.zeros((num_objects, k_i))
                    newcol[i,:] = 1 
                    Z_temp = np.column_stack((Z_temp, newcol))
                trunc[k_i] = k_i * np.log(alpha_N) - alpha_N - np.log(np.math.factorial(k_i)) + likelihood_opt1(data, Z_temp,sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

            trunc = np.exp(trunc - np.max(trunc))
            trunc = trunc/np.sum(trunc)

            p = np.random.uniform(0,1)
            t = 0
            new_dishes = 0

            for k_i in range(0,5):
                t = t + trunc[k_i]
                if p < t:
                    new_dishes = k_i
                    break

            if(new_dishes > 0):
                newcol = np.zeros((num_objects, new_dishes))
                newcol[i,:] = 1
                Z = np.column_stack((Z, newcol))
            K_plus = K_plus + new_dishes

        #Sample sigma_X and sigma_A through Metropolis
        lik_curr = likelihood_opt1(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim)

        if np.random.uniform(0,1) < 0.5:
            sigma_X_new = sigma_X - np.random.uniform(0,1)/20
        else:
            sigma_X_new = sigma_X + np.random.uniform(0,1)/20

        lik_new_X = likelihood_opt1(data, Z, sigma_A, sigma_X_new, K_plus, num_objects, object_dim)

        acc_X = np.exp(min(0, lik_new_X - lik_curr))

        if np.random.uniform(0,1) < 0.5:
            sigma_A_new = sigma_A - np.random.uniform(0,1)/20
        else:
            sigma_A_new = sigma_A + np.random.uniform(0,1)/20

        lik_new_A = likelihood_opt1(data, Z, sigma_A_new, sigma_X, K_plus, num_objects, object_dim)

        acc_A = np.exp(min(0, lik_new_A - lik_curr))

        if np.random.uniform(0,1) < acc_X:
            sigma_X = sigma_X_new
        if np.random.uniform(0,1) < acc_A:
            sigma_A = sigma_A_new

        #Sample alpha via Gibbs
        alpha = gamma.rvs(a = 1 + K_plus, scale = 1/(1+HN))

    print("Complete")
    return(chain_K)

/opt/conda/lib/python3.4/site-packages/IPython/utils/path.py:264: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module
  warn("get_ipython_cache_dir has moved to the IPython.paths module")


In [92]:
%%timeit
G = sampler(image_data, num_objects, object_dim)

0
('At iteration', 0, ': K_plus is', 2, ', alpha is', 1.0)
('At iteration', 1, ': K_plus is', 1, ', alpha is', 1.539014447414358)
('At iteration', 2, ': K_plus is', 1, ', alpha is', 1.483966881251176)
('At iteration', 3, ': K_plus is', 1, ', alpha is', 1.3008188378175622)
('At iteration', 4, ': K_plus is', 1, ', alpha is', 0.2803837446795034)
('At iteration', 5, ': K_plus is', 1, ', alpha is', 0.2650613379769046)
('At iteration', 6, ': K_plus is', 1, ', alpha is', 1.6469456384940555)
('At iteration', 7, ': K_plus is', 1, ', alpha is', 0.9392554177150969)
('At iteration', 8, ': K_plus is', 1, ', alpha is', 1.1889613834469839)
('At iteration', 9, ': K_plus is', 1, ', alpha is', 1.686895771287755)
('At iteration', 10, ': K_plus is', 1, ', alpha is', 0.6855439856232375)
('At iteration', 11, ': K_plus is', 1, ', alpha is', 1.3616093654971666)
('At iteration', 12, ': K_plus is', 1, ', alpha is', 0.9581715887316585)
('At iteration', 13, ': K_plus is', 1, ', alpha is', 1.5416179949888698)
('At

Using JIT
====

In [72]:
@jit
## Optimized likelihood
# This function return the log likelihood
def likelihood_opt2(X, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    #Calculate M
    M = np.dot(Z.T, Z) + (sigma_X**2/sigma_A**2)*np.eye(K_plus)
    
    part1 = (-1)*num_objects*(0.5*object_dim)*np.log(2*np.pi)
    part2 = (-1)*(num_objects-K_plus)* object_dim *np.log(sigma_X) 
    part3 = (-1)*object_dim*K_plus*np.log(sigma_A) 
    part4 = (-1)*(0.5*object_dim)* np.log(np.linalg.det(M)) 
    part5 = (-1/(2*sigma_X**2)) * np.trace(np.dot(np.dot(X.T,(np.identity(num_objects) - np.dot(np.dot(Z,np.linalg.inv(M)),Z.T))),X))
    total = part1+part2+part3+part4+part5
    return(total)

In [73]:
%%timeit

data = image_data

#Set number of iterations
E = 10

#Set truncation limit for max number of sampled latent features
K_inf = 20

#Set storage arrays for sampled parameters
chain_Z = np.zeros([E, num_objects, K_inf])
chain_K = np.zeros([E, 1])
chain_sigma_X = np.zeros([E, 1])
chain_sigma_A = np.zeros([E, 1])
chain_alpha = np.zeros([E, 1])

#Initialize parameter values
sigma_X = 1
sigma_A = 1
alpha = 1
num_object= np.shape(data)[0]
object_dim = np.shape(data)[1]
[Z, K_plus] = sampleIBP(alpha, num_objects)

#Compute Harmonic Number
HN = 0
for i in range(0, num_objects):
    HN = HN + 1/(i+1)

for e in range(0, E):

    #Store sampled values
    chain_Z[e, :, 0:K_plus] = Z[:, 0:K_plus]
    chain_K[e] = K_plus
    chain_sigma_X[e] = sigma_X
    chain_sigma_A[e] = sigma_A
    chain_alpha[e] = alpha
    
    if (e%100==0):
        print(e)
    
    print("At iteration", e, ": K_plus is", K_plus, ", alpha is", alpha) 

    #Generate a new value for Z[i,k] and accept by Metropolis
    for i in range(0, num_objects):
        #First we remove singular features if any
        for k in range(0, K_plus):
            if (k>=K_plus):
                break
            if(Z[i, k] > 0):
                if (np.sum(Z[:, k]) - Z[i, k]) <= 0: 
                    Z[i, k] = 0
                    Z[:, k:(K_plus - 1)] = Z[:, (k+1):K_plus]
                    K_plus = K_plus - 1
                    Z = Z[:, 0:K_plus]
                    continue
            #Compute conditional distribution for current cell
            P=np.zeros(2)

            Z[i,k]=1
            P[0] = likelihood_opt2(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(np.sum(Z[:, k]) - Z[i,k]) - np.log(num_objects)

            Z[i,k]=0
            P[1] = likelihood_opt2(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - np.sum(Z[:,k])) - np.log(num_objects)

            P = np.exp(P - np.max(P))

            #Sample from conditional(sampling previously sampled dishes)
            if np.random.uniform(0,1) < (P[0]/(np.sum(P))):
                Z[i, k] = 1
            else:
                Z[i, k] = 0
                
        #Sample new dishes by Metropolis
        trunc = np.zeros(5)
        alpha_N = alpha/num_objects

        for k_i in range(0,5):
            Z_temp = Z
            if k_i>0:
                newcol = np.zeros((num_objects, k_i))
                newcol[i,:] = 1 
                Z_temp = np.column_stack((Z_temp, newcol))
            trunc[k_i] = k_i * np.log(alpha_N) - alpha_N - np.log(np.math.factorial(k_i)) + likelihood_opt2(data, Z_temp,sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

        trunc = np.exp(trunc - np.max(trunc))
        trunc = trunc/np.sum(trunc)

        p = np.random.uniform(0,1)
        t = 0
        new_dishes = 0

        for k_i in range(0,5):
            t = t + trunc[k_i]
            if p < t:
                new_dishes = k_i
                break

        if(new_dishes > 0):
            newcol = np.zeros((num_objects, new_dishes))
            newcol[i,:] = 1
            Z = np.column_stack((Z, newcol))
        K_plus = K_plus + new_dishes

    #Sample sigma_X and sigma_A through Metropolis
    lik_curr = likelihood_opt2(data, Z, sigma_A, sigma_X, K_plus, num_objects, object_dim)

    if np.random.uniform(0,1) < 0.5:
        sigma_X_new = sigma_X - np.random.uniform(0,1)/20
    else:
        sigma_X_new = sigma_X + np.random.uniform(0,1)/20

    lik_new_X = likelihood_opt2(data, Z, sigma_A, sigma_X_new, K_plus, num_objects, object_dim)

    acc_X = np.exp(min(0, lik_new_X - lik_curr))

    if np.random.uniform(0,1) < 0.5:
        sigma_A_new = sigma_A - np.random.uniform(0,1)/20
    else:
        sigma_A_new = sigma_A + np.random.uniform(0,1)/20

    lik_new_A = likelihood_opt2(data, Z, sigma_A_new, sigma_X, K_plus, num_objects, object_dim)

    acc_A = np.exp(min(0, lik_new_A - lik_curr))

    if np.random.uniform(0,1) < acc_X:
        sigma_X = sigma_X_new
    if np.random.uniform(0,1) < acc_A:
        sigma_A = sigma_A_new

    #Sample alpha via Gibbs
    alpha = np.random.gamma(1 + K_plus, 1/(1+HN))

print("Complete")

0
At iteration 0 : K_plus is 6 , alpha is 1
At iteration 1 : K_plus is 3 , alpha is 0.35709921757350666
At iteration 2 : K_plus is 2 , alpha is 0.4117725790572868
At iteration 3 : K_plus is 1 , alpha is 0.15716635130447076
At iteration 4 : K_plus is 1 , alpha is 0.15365446515654269
At iteration 5 : K_plus is 1 , alpha is 0.4830313363766117
At iteration 6 : K_plus is 1 , alpha is 0.25791381871630964
At iteration 7 : K_plus is 1 , alpha is 0.40729661475817813
At iteration 8 : K_plus is 1 , alpha is 0.8697804290384793
At iteration 9 : K_plus is 1 , alpha is 0.5806042254380704
Complete
0
At iteration 0 : K_plus is 6 , alpha is 1
At iteration 1 : K_plus is 5 , alpha is 0.44379280061073384
At iteration 2 : K_plus is 4 , alpha is 0.5431456109625356
At iteration 3 : K_plus is 3 , alpha is 0.2168788178235729
At iteration 4 : K_plus is 3 , alpha is 0.8772914339218634
At iteration 5 : K_plus is 3 , alpha is 1.436574398924792
At iteration 6 : K_plus is 3 , alpha is 0.6013603193567629
At iteration 